In [ ]:
from datasets import load_dataset
import re, os, json

In [ ]:
dataset = load_dataset("json", data_files="train.jsonl")
for d in dataset['train']['question'][:2]:
    print(d.split()[0])

In [ ]:
os.makedirs("TTA_scaling", exist_ok=True)
def has_final_consonant(char):
    code = ord(char)
    if 0xAC00 <= code <= 0xD7A3:
        jong = (code - 0xAC00) % 28
        return jong != 0
    return False

print(has_final_consonant("가"))  # False (받침 없음)
print(has_final_consonant("각"))  # True (받침 있음)

In [ ]:
folder_path = 'TTA'
folder_list = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
for folder in folder_list:
    foldername = folder.split('_')
    scaling_foldername = "TTA_scaling/" + ('_'.join(foldername[:-1] + ['scaling', foldername[-1]]))
    os.makedirs(scaling_foldername, exist_ok=True)
    print(scaling_foldername)
    file_list = os.listdir('/'.join([folder_path, folder]))
    
    for file in file_list:
        print(file.split('.')[0])
        if(file.split('.')[-1] != '.json'): pass
        data_files= '/'.join([folder_path, folder, file])
        with open(data_files, "r", encoding="utf-8") as f:
            datas = json.load(f)
        
        
        scaling_data_files = '/'.join([scaling_foldername, file.split('.')[0] + '.jsonl'])
        print(scaling_data_files)
        with open(scaling_data_files, 'w', encoding="utf-8") as f:
            for data in datas:
                for key, value in data.items():
                    question = (key + ('이란' if has_final_consonant(key[-1]) else '란') + ' 무엇인가?')
                    qa = {"question": question, "answer": value}
                    print(qa)
                    f.write(json.dumps(qa, ensure_ascii=False) + "\n")
        datas